In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, classification_report
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy

In [2]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [3]:
X = train_data['text'].tolist()
y = train_data['target'].tolist()

# **Tokenizing the text**

In [4]:
max = 1000
tokenizer = Tokenizer(num_words = max, oov_token='<OOV>')
tokenizer.fit_on_texts(X)
sequence = tokenizer.texts_to_sequences(X)

In [5]:
maxlen = 100
dataX =pad_sequences(sequence, maxlen=maxlen)

In [6]:
y = np.array(y)

In [7]:
X_train, X_val, y_train, y_val = train_test_split(dataX, y, test_size=0.2, random_state=42)

In [8]:
model = Sequential()
model.add(Embedding(input_dim=max, output_dim=128, input_length=maxlen))
model.add(LSTM(units=64))
model.add(Dense(units=1, activation='sigmoid'))

In [9]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [10]:
early_stopping = EarlyStopping(monitor='val_loss', patience=4,verbose=1,restore_best_weights = True)

In [11]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/20
191/191 [==============================] - 22s 98ms/step - loss: 0.5492 - accuracy: 0.7169 - val_loss: 0.4720 - val_accuracy: 0.7892
Epoch 2/20
191/191 [==============================] - 16s 83ms/step - loss: 0.4155 - accuracy: 0.8136 - val_loss: 0.4557 - val_accuracy: 0.7945
Epoch 3/20
191/191 [==============================] - 16s 85ms/step - loss: 0.3817 - accuracy: 0.8371 - val_loss: 0.4720 - val_accuracy: 0.8004
Epoch 4/20
191/191 [==============================] - 16s 85ms/step - loss: 0.3605 - accuracy: 0.8470 - val_loss: 0.4808 - val_accuracy: 0.7807
Epoch 5/20
191/191 [==============================] - 17s 91ms/step - loss: 0.3392 - accuracy: 0.8571 - val_loss: 0.5078 - val_accuracy: 0.7866
Epoch 6/20
191/191 [==============================] - 16s 86ms/step - loss: 0.3201 - accuracy: 0.8632 - val_loss: 0.5215 - val_accuracy: 0.7768
Epoch 6: early stopping


In [12]:
model.save('trained_lstm_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
